In [1]:
import pandas as pd

from features import analytical_tools
from utils import helper_funtions

ImportError: DLL load failed while importing _path: No se puede encontrar el módulo especificado.

In [ ]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes

In [ ]:
PATH = helper_funtions.get_path()

# Read data

In [ ]:
df = pd.read_csv(PATH + 'data/data_cleaning.csv')
df_validation = pd.read_csv(PATH + 'data/data_validation.csv')

df.sample(n=5).head()

# Preprocesing

In [ ]:
# Determine peak times by rounding the time in groups of one hour 
df['Contact_hour'] = pd.to_datetime(df['Contact_hour'], format='%H:%M:%S')
df['Contact_hour_round'] = df['Contact_hour'].apply(lambda x: x.round('60 min'))

df_peak_time = df['Contact_hour_round'].value_counts().head()
df_peak_time = df_peak_time.rename_axis('Hour')
df_peak_time = df_peak_time.reset_index(name='Count')
peak_hours = []
for i in range(len(df_peak_time)):
    peak_hours.append(df_peak_time['Hour'].iloc[i])

for i in range(len(df)):
    if df.loc[i, 'Contact_hour_round'] in peak_hours:
        df.at[i, 'Is_peak_hour'] = 1
    else:
        df.at[i, 'Is_peak_hour'] = 0

del df_peak_time

In [ ]:
# Determine peak times by rounding the time in groups of one hour 
df_validation['Contact_hour'] = pd.to_datetime(df_validation['Contact_hour'], format='%H:%M:%S')
df_validation['Contact_hour_round'] = df_validation['Contact_hour'].apply(lambda x: x.round('60 min'))

df_peak_time = df_validation['Contact_hour_round'].value_counts().head()
df_peak_time = df_peak_time.rename_axis('Hour')
df_peak_time = df_peak_time.reset_index(name='Count')
peak_hours = []
for i in range(len(df_peak_time)):
    peak_hours.append(df_peak_time['Hour'].iloc[i])

for i in range(len(df_validation)):
    if df_validation.loc[i, 'Contact_hour_round'] in peak_hours:
        df_validation.at[i, 'Is_peak_hour'] = 1
    else:
        df_validation.at[i, 'Is_peak_hour'] = 0

del df_peak_time

In [ ]:
# Obtener el porcentaje de impuesto
df['TAX_percent'] = (df['TAX'].mul(100) / df['Income']).round(0)
df['TAX_percent'].value_counts()

# Simple imputation

In [ ]:
df_new = df.query("Age < 70")
mean_age = df_new['Age'].mean()

index = df.query("Age > 70").index[0]

df.at[index, 'Age'] = mean_age

In [ ]:
df_new = df.query("Previous_sales > 0")
mean_previous_sales = df_new['Previous_sales'].mean()

indexes = []
for i in range(0, len(df.query("Previous_sales < 0").index)):
    indexes.append(df.query("Previous_sales < 0").index[i])

for index in indexes:
    df.at[index, 'Previous_sales'] = mean_previous_sales

In [ ]:
len(df.query("Num_contacts > 5"))

# Label encoders

In [ ]:
labels_to_encoders_name = {
    'Contact_channel':'Contact_channel_ENC',
    'Type_products':'Type_products_ENC',
    'Contact_hour_round':'Contact_hour_round_ENC'}

for var_encoder, encoder_name in labels_to_encoders_name.items():
    analytical_tools.label_encoder(var_encoder, encoder_name, df)
    analytical_tools.label_encoder(var_encoder, encoder_name, df_validation)

In [ ]:
df[['Contact_channel','Contact_channel_ENC',
    'Type_products','Type_products_ENC',
    'Contact_hour_round','Contact_hour_round_ENC']].head(5)

In [ ]:
df_validation[['Contact_channel','Contact_channel_ENC',
    'Type_products','Type_products_ENC',
    'Contact_hour_round','Contact_hour_round_ENC']].head(5)

# Model

In [ ]:
from pycaret.classification import *
data = df.sample(frac=0.8, random_state=0)
data.head()

In [ ]:
data_unseen = df.drop(data.index)
data_unseen.head()

In [ ]:
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)


# Save

In [ ]:
# df.to_csv(PATH + 'data/data_preprocesing.csv', index=False, encoding='utf-8')

In [ ]:
df.sample(n=5).head()